In [2]:
import pandas as pd
import numpy as np

import os
import glob

import re

pd.options.display.max_columns = None

## **Data Understanding**

Ini adalah dataset publik e-commerce Brasil dari pesanan yang dibuat di Olist Store. Dataset ini memiliki informasi dari 100 ribu pesanan dari tahun 2016 hingga 2018 yang dibuat di beberapa pasar di Brasil. Fitur-fiturnya memungkinkan untuk melihat pesanan dari berbagai dimensi: mulai dari status pesanan, harga, pembayaran, dan kinerja pengiriman hingga lokasi pelanggan, atribut produk, dan akhirnya ulasan yang ditulis oleh pelanggan. Selain itu ada juga dataset geolokasi yang menghubungkan kode pos Brasil dengan koordinat lintang/lintang.

### **Data Schema**

Data dibagi dalam beberapa dataset. Skema data bisa dilihat pada gambar berikut:
<p> <p>
<figure>
  <center><img src = "https://raw.githubusercontent.com/aridiawan/ecommerce-analysis/main/data/raw/ERD.jpg" alt ="Scheme Data Table" width = 900; height = 647></center>
  <figcaption align = "center">Gambar 1. <em>Scheme Data</em> </figcaption>
</figure>
<p> <p>

### **Feature Description**

Berikut adalah fitur-fitur yang tersedia dalam dataset:
<p> <p>
<figure>
  <center><img src = "https://raw.githubusercontent.com/aridiawan/ecommerce-analysis/main/data/raw/Olist-Dataset-Description.jpg" alt ="Scheme Data Table" width = 900; height = 1056></center>
  <figcaption align = "center">Gambar 2. <em>Deskripsi Feature</em> </figcaption>
</figure>
<p> <p>

### **Data Skimming**

In [3]:
# load data
mydir = r"D:\PURWADHIKA\FINAL PROJECT\ecommerce-analysis\data\raw"

file_list = glob.glob(mydir + "\*.csv")

In [4]:
# function text handling

def efn(text):
    file = text.split('\\')[-1].replace('.csv','')
    return file

def white_string_to_nan(data):
    return data.replace(r'^\s*$', np.nan, regex=True)

In [5]:
# create dictinoary of dataset

dfs = {f'{efn(file)}_df': white_string_to_nan(pd.read_csv(file, engine="pyarrow")) for file in file_list}

In [6]:
# check data info

dataInfo = []

for n in dfs.keys():
    dataInfo.append({
        'dataset_name' : n,
        'shape' : dfs[n].shape,
        'columns' : list(dfs[n].columns),
        'duplicate' : dfs[n].duplicated().sum(),
        'null' : dfs[n].isna().sum().sum(),
        'null_columns' : dfs[n].columns[dfs[n].isna().any()].tolist()
    })

pd.DataFrame(dataInfo)


,dataset_name,shape,columns,duplicate,null,null_columns
0,olist_customers_dataset_df,"(99441, 5)","[customer_id, customer_unique_id, customer_zip...",0,0,[]
1,olist_geolocation_dataset_df,"(1000163, 5)","[geolocation_zip_code_prefix, geolocation_lat,...",261831,0,[]
2,olist_orders_dataset_df,"(99441, 8)","[order_id, customer_id, order_status, order_pu...",0,4908,"[order_approved_at, order_delivered_carrier_da..."
3,olist_order_items_dataset_df,"(112650, 7)","[order_id, order_item_id, product_id, seller_i...",0,0,[]
4,olist_order_payments_dataset_df,"(103886, 5)","[order_id, payment_sequential, payment_type, p...",0,0,[]
5,olist_order_reviews_dataset_df,"(99224, 7)","[review_id, order_id, review_score, review_com...",0,145932,"[review_comment_title, review_comment_message]"
6,olist_products_dataset_df,"(32951, 9)","[product_id, product_category_name, product_na...",0,2448,"[product_category_name, product_name_lenght, p..."
7,olist_sellers_dataset_df,"(3095, 4)","[seller_id, seller_zip_code_prefix, seller_cit...",0,0,[]
8,product_category_name_translation_df,"(71, 2)","[product_category_name, product_category_name_...",0,0,[]


In [7]:
# skimming function

def skim(data, name):
    summary = pd.DataFrame({
                'dataset' : name,
                'column': data.columns.values,
                'type': data.dtypes.values,
                'n_unique': data.nunique().values,
                'n_missing': data.isna().sum().reset_index()[0],
                'pct_missing': round(data.isna().sum().reset_index()[0]/len(data)*100, 2),
                'min': data.dropna().min().values,
                'max': data.dropna().max().values
                })
    return summary

In [8]:
# check

summ = pd.concat([skim(dfs[n], n) for n in dfs.keys()], sort=False)
summ

,dataset,column,type,n_unique,n_missing,pct_missing,min,max
0,olist_customers_dataset_df,customer_id,object,99441,0,0.00,00012a2ce6f8dcda20d059ce98491703,ffffe8b65bbe3087b653a978c870db99
1,olist_customers_dataset_df,customer_unique_id,object,96096,0,0.00,0000366f3b9a7992bf8c76cfdf3221e2,ffffd2657e2aad2907e67c3e9daecbeb
2,olist_customers_dataset_df,customer_zip_code_prefix,int64,14994,0,0.00,1003,99990
3,olist_customers_dataset_df,customer_city,object,4119,0,0.00,abadia dos dourados,zortea
4,olist_customers_dataset_df,customer_state,object,27,0,0.00,AC,TO
0,olist_geolocation_dataset_df,geolocation_zip_code_prefix,int64,19015,0,0.00,1001,99990
1,olist_geolocation_dataset_df,geolocation_lat,float64,717372,0,0.00,-36.605374,45.065933
2,olist_geolocation_dataset_df,geolocation_lng,float64,717615,0,0.00,-101.466766,121.105394
3,olist_geolocation_dataset_df,geolocation_city,object,8011,0,0.00,* cidade,óleo
4,olist_geolocation_dataset_df,geolocation_state,object,27,0,0.00,AC,TO


**olist_customers_dataset_df**

**olist_geolocation_dataset_df**

**olist_orders_dataset_df**

In [11]:
order = dfs['olist_orders_dataset_df']
order['order_status'].unique()

array(['delivered', 'invoiced', 'shipped', 'processing', 'unavailable',
       'canceled', 'created', 'approved'], dtype=object)

**olist_order_items_dataset_df**

In [15]:
order_item = dfs['olist_order_items_dataset_df']
order_item['order_item_id'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21], dtype=int64)

**olist_order_payments_dataset_df**

**olist_order_reviews_dataset_df**

**olist_products_dataset_df**

**olist_sellers_dataset_df**

**product_category_name_translation_df**